# Minigame 12a: Evaluation on a multi-resolution grid

In this exploration, we're looking at the ability to take policies trained on small patches and to apply them to larger meshes by composition - but in this case we want to be able to handle a grid that already has different levels of resolution present.

In [1]:
# L2 sum bugfix in training
chkptfile = "/home/rwa/ray_results/PPO_EstimatorGame_2021-02-27_18-57-57nhfajmzk/checkpoint_75/checkpoint-75"

# All of above and sin waves in library
chkptfile="/home/rwa/ray_results/PPO_EstimatorGame_2021-02-27_20-58-32bp5n1ofm/checkpoint_115/checkpoint-115"


In [2]:
import math
from math import sin,cos
import random

In [3]:
import gym
from gym import spaces, utils
import numpy as np
import ray
import ray.rllib.agents.ppo as ppo
import ray.rllib.agents.dqn as dqn

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from glvis import glvis, to_stream
from ipywidgets import Layout

In [5]:
import matplotlib.pyplot as plt

In [6]:
from mfem import path
import mfem.ser as mfem

Define some synthetic test functions.

In [7]:
def rotate(x,theta):
    x0 = x[0]
    y0 = x[1]
    x1 = x0*cos(theta)-y0*sin(theta)
    y1 = x0*sin(theta)+y0*cos(theta)
    return [x1,y1]

In [8]:
def bump(x):
    rsq = x[0]**2 +x[1]**2
    return math.exp(-rsq)

In [9]:
def step(x):
    x0 = x[0]
    if (x0 < 0.0):
        return 1.0
    else:
        return 0.0

In [10]:
def wave(x):
    return sin(2.*math.pi*x[0])

In [11]:
def smooth_step(x):
    return 0.5*(1.0 +math.tanh(x[0]))

In [12]:
def rotated_smooth_step(x,theta):
    xr = rotate(x,theta)
    return smooth_step(xr)

In [13]:
def rotated_step(x, theta):
    xr = rotate(x,theta)
    return step(xr)

In [14]:
def ramp(x):
    if x[0] < 0.5:
        return 0.5*x[0]+0.25
    else:
        return 0.25+0.25

Create classes where we can set the parameters and then eval a bunch of points.

In [15]:
class Step(mfem.PyCoefficient):
    
    def SetParams(self):
        self.theta = random.uniform(0.0, 2.0*math.pi)
        self.dx = [random.uniform(-0.5, 0.5),random.uniform(-0.5, 0.5)]
        self.xc = [0.5,0.5]
    def EvalValue(self, x):
        return rotated_step(x-self.xc+self.dx, self.theta)

In [16]:
class Wave(mfem.PyCoefficient):
    
    def SetParams(self):
        self.theta = random.uniform(0.0, 2.0*math.pi)
        self.dx = [random.uniform(-0.5, 0.5),random.uniform(-0.5, 0.5)]
        self.ampl = random.uniform(0.0, 1.0)
        self.wavel = random.uniform(0.0, 1.0)
        
    def EvalValue(self, x):
        x *= self.wavel
        x -= self.dx
        xr = rotate(x,self.theta)
        return self.ampl*wave(xr)

In [17]:
class Linear(mfem.PyCoefficient):
    
    def SetParams(self):
        self.theta = random.uniform(0.0, 2.0*math.pi)
        self.y0 = random.uniform(0.0,1.0)
        self.y1 = random.uniform(0.0,1.0)
        
        self.m = self.y1-self.y0
        self.b = 0.5*(self.y0+self.y1)
        
    def EvalValue(self, x):
        xc = x-[0.5,0.5]
        xr = rotate(xc,self.theta)
        line = self.m*xr[0]+self.b
        return line

In [18]:
class Ramp(mfem.PyCoefficient):
    
    def SetParams(self):
        pass
        
    def EvalValue(self, x):
        return ramp(x)

In [19]:
class Bump(mfem.PyCoefficient):
    
    def SetParams(self):
        self.width = [random.uniform(0.01,0.5),random.uniform(0.01,0.5)]
        self.xc = [0.5,0.5]
        self.dx = [random.uniform(-0.5, 0.5),random.uniform(-0.5, 0.5)]

    def EvalValue(self, x):
        return bump((x-self.xc+self.dx)/self.width)

In [20]:
class SmoothStep(mfem.PyCoefficient):
    
    def SetParams(self):
        self.width = random.uniform(1.0, 15.0)
        self.xc = [0.5,0.5]
        self.dx = random.uniform(-0.5,0.5)
        self.theta = random.uniform(0.0, 2.0*math.pi)

    def EvalValue(self, x):
        x -= self.xc
        x += self.dx
        return rotated_smooth_step(x*self.width, self.theta)

In [21]:
class BumpAndSmoothStep(mfem.PyCoefficient):
    
    def SetParams(self):
        self.bump = Bump()
        self.bump.SetParams()
        self.smooth_step = SmoothStep()
        self.smooth_step.SetParams()

    def EvalValue(self, x):
        return 0.5*self.bump.EvalValue(x)+0.5*self.smooth_step.EvalValue(x)

In [22]:
class Library(mfem.PyCoefficient):
    
    def SetParams(self):
        # 0 - linear
        # 1 - bump
        # 2 - tanh
        # 3 - step
        # 4 - wave
        pick = random.randrange(4)
        if pick == 0:
            self.fn = Linear()
        elif pick == 1:
            self.fn = Bump()
        elif pick == 2:
            self.fn = SmoothStep()
        elif pick == 3:
            self.fn = Step()
        elif pick == 4:
            self.fn = Wave()
        
        self.fn.SetParams()

    def EvalValue(self, x):
        return self.fn.EvalValue(x)

In [89]:
class LinearCombination(mfem.PyCoefficient):
    
    def SetParams(self):
        self.n = 3
        self.fn = []
            # 0 - linear
            # 1 - bump
            # 2 - tanh
            # 3 - step
            # 4 - wave
        for i in range(self.n):
            pick = random.randrange(3)
            if pick == 0:
                self.fn.append(Linear())
            elif pick == 1:
                self.fn.append(Bump())
            elif pick == 2:
                self.fn.append(SmoothStep())
            elif pick == 3:
                self.fn.append(Step())
            self.fn[-1].SetParams()
        
    def EvalValue(self, x):
        usum = 0.0
        for i in range(self.n):
            usum += self.fn[i].EvalValue(x)
        return usum/self.n

Visualize an instance of the test function. Note that each instance has randomly chosen parameters.  For the steps, it's a rotation angle and a displacement.  For the bumps, it's a width and a displacement.

In [90]:
class EstimatorGameDummy(gym.Env):
    
        
    # In RLlib, you need the config arg
    def __init__(self,config):
        
        # keep a copy of the unrefined mesh so we can restore it from memory
        self.meshfile = 'inline-quad-5.mesh'
        self.mesh0 = mfem.Mesh(self.meshfile)
        self.mesh = mfem.Mesh(self.meshfile)
        self.width = math.sqrt(self.mesh.GetNE())
        
        # The only reason we need to create an fespace and gf here
        # is to find the sizes needed for the action and observation spaces
        dim = self.mesh.Dimension()
        self.order = 1
        self.fec = mfem.L2_FECollection(self.order, dim)
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)
        self.u = mfem.GridFunction(self.fes);

        # actions are: do nothing (0) or refine center element (1)
        self.action_space = spaces.Discrete(2)
        
        self.obsx = 42
        self.obsy = 42
        
        # observation space: 42x42 image
        self.observation_space = spaces.Box(-1.0, 2.0, shape=(self.obsx,self.obsy,1))
        #self.get_obs_points()
    
    def step(self, action):
        pass
    
    # similar to reset, but do not choose a new function
    def reinit(self):
        #print("reinit")
        del self.mesh
        self.mesh = mfem.Mesh(self.mesh0)

        del self.fes
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)

        del self.u
        self.u = mfem.GridFunction(self.fes)
        self.u.ProjectCoefficient(self.u0)
        
        #self.get_obs_points()
        obs = self.get_obs()
        
        return np.array(obs)
    
    # every reset of the env chooses a new synthetic function
    def reset(self):
        self.u0 = self.u0_coeff()
        self.u0.SetParams()
        return self.reinit()
    
    def render(self):
        return glvis(to_stream(self.mesh,self.u) + 'keys Rjlmc',600,600)

In [91]:
#env = EstimatorGameDummy(None)
#env.reset()
#env.render()

Now we want to load a trained policy, and apply it in a strided way.

In [92]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

2021-03-04 20:30:33,255	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.201',
 'raylet_ip_address': '192.168.1.201',
 'redis_address': '192.168.1.201:6379',
 'object_store_address': '/tmp/ray/session_2021-03-04_20-30-32_746944_29772/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-03-04_20-30-32_746944_29772/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-03-04_20-30-32_746944_29772',
 'metrics_export_port': 55652,
 'node_id': '0b0dc85a4febf68399d018ad7fa5a469245451718a203bcf89ac1c28'}

In [93]:
ppo_config = ppo.DEFAULT_CONFIG.copy()
dqn_config = dqn.DEFAULT_CONFIG.copy()

#ppo_config['framework'] = 'tfe'
#dqn_config['num_workers'] = 1

ppo_config['train_batch_size'] = 1000
ppo_config['num_workers'] = 3
ppo_config['num_gpus'] = 0

agent = ppo.PPOTrainer(ppo_config, env=EstimatorGameDummy)

#agent = dqn.DQNTrainer(dqn_config, env=AMRGameDummy)
ppo_config

(pid=6737) WARNING:tensorflow:From /home/rwa/minigame_tf1/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6737) Instructions for updating:
(pid=6737) non-resource variables are not supported in the long term
(pid=6737) WARNING:tensorflow:From /home/rwa/minigame_tf1/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6737) Instructions for updating:
(pid=6737) non-resource variables are not supported in the long term
(pid=6739) WARNING:tensorflow:From /home/rwa/minigame_tf1/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6739

2021-03-04 20:30:38,444	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-03-04 20:30:41,815	WARNING util.py:47 -- Install gputil for GPU system monitoring.


{'num_workers': 3,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': 1000,
 'model': {'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': False,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,
  'attention_init_gru_gate_bias': 2.0,
  'num_framestacks': 'auto',
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_model_config': {},
  'custom_actio

Restore a policy

Long trained on BumpsAndSmoothSteps

In [94]:
#agent.restore("/home/rwa/ray_results/PPO_EstimatorGame_2021-02-26_22-33-45kb_md4j2/checkpoint_540/checkpoint-540")

Trained on Library, with linear functions

In [95]:
agent.restore(chkptfile)

2021-03-04 20:30:42,048	INFO trainable.py:372 -- Restored on 192.168.1.201 from checkpoint: /home/rwa/ray_results/PPO_EstimatorGame_2021-02-27_20-58-32bp5n1ofm/checkpoint_115/checkpoint-115
2021-03-04 20:30:42,048	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 115, '_timesteps_total': None, '_time_total': 10348.874559640884, '_episodes_total': 138000}


In [96]:
policy = agent.get_policy()

Now we want to create the larger problem we'll be applying this local indicator on.

In [97]:
class GlobalProblem:

    def __init__(self,env):
        self.mesh = mfem.Mesh('star.mesh')
        self.mesh.UniformRefinement()
        self.mesh.UniformRefinement()

        self.mesh0 = mfem.Mesh(self.mesh)
        self.dim = 2
        self.p = 1
        self.fec = mfem.L2_FECollection(self.p, self.dim)
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)
        self.u = mfem.GridFunction(self.fes)
        self.u0 = mfem.GridFunction(self.fes)
        self.coeff = LinearCombination()
        self.coeff.SetParams()
        self.u.ProjectCoefficient(self.coeff)
        self.u0.Assign(self.u) # save so we can restore later if desired
        self.env = env
        self.obsx = 42
        self.obsy = 42
        
    def reset(self):
        self.coeff.SetParams()
        self.reinit()
        
    def reinit(self):
        del self.mesh
        self.mesh = mfem.Mesh(self.mesh0)

        del self.fes
        self.fes = mfem.FiniteElementSpace(self.mesh, self.fec)

        del self.u
        self.u = mfem.GridFunction(self.fes)
        self.u.ProjectCoefficient(self.coeff)
        
    # compute observation points around element k
    def compute_observation(self, k):
        # build the matrix of sample points
        ref_w = 2.0
        ref_n = self.obsx
        assert(self.obsx == self.obsy)
        ref_lo = -ref_w
        ref_hi = +ref_w
        ref_dx = (ref_hi-ref_lo)/ref_n
    
        trk = self.mesh.GetElementTransformation(k)
        ipk = mfem.IntegrationPoint()
        xk = mfem.Vector(self.dim)

        m = np.empty((ref_n*ref_n,self.dim))
        k = 0
        for j in range(ref_n):
            ipk.y = ref_lo +(j+0.5)*ref_dx
            for i in range(ref_n):
                ipk.x = ref_lo +(i+0.5)*ref_dx
                xk = trk.Transform(ipk)
                m[k,:] = xk
                k += 1
        ns, self.sample_els, self.sample_ips = self.mesh.FindPoints(m)
        
        state = np.empty((self.obsx,self.obsy,1))
        k = 0
        complete = True
        for j in range(self.obsy):
            for i in range(self.obsx):
                el = self.sample_els[k]
                if self.sample_els[k] >= 0:
                    ip = self.sample_ips[k]
                    v = self.u.GetValue(self.sample_els[k],self.sample_ips[k])
                    state[i][j] = v
                else:
                    state[i][j] = 0.0
                    complete = False
                k += 1
        self.state = state
        return state, complete     
        
    def refine(self):
        
        # algorithm:
        # foreach element:
        #   generate sample points
        #   sample the function to create the obs
        #   compute the policy action=pi(obs)
        #   record the elements to be refined
        
        self.l2_0_fec = mfem.L2_FECollection(0,self.dim)
        self.l2_0_fes = mfem.FiniteElementSpace(self.mesh,self.l2_0_fec)
        self.complete = mfem.GridFunction(self.l2_0_fes);
        
        refine_list = []
        for k in range(self.mesh.GetNE()):
            obs, complete = self.compute_observation(k)
            #complete = True # temp
            self.complete[k] = complete
            if complete:
                action, _, info = policy.compute_single_action(obs, explore=False)
                if action == 1:
                    refine_list.append(k)
        
        if len(refine_list) > 0:
            self.mesh.GeneralRefinement(mfem.intArray(refine_list))
            self.fes.Update()
            self.u.Update()
            self.u.ProjectCoefficient(self.coeff)
            
            self.l2_0_fes.Update()
            self.complete.Update()
            
    def refine_by_dg(self):
        # put the L2 gridfunction into a coefficient so we can project it
        u_disc_coeff = mfem.GridFunctionCoefficient(self.u)
        h1_fec = mfem.H1_FECollection(self.p, self.dim)
        h1_fes = mfem.FiniteElementSpace(self.mesh, h1_fec)
        u_h1 = mfem.GridFunction(h1_fes)
        u_h1.ProjectDiscCoefficient(u_disc_coeff, mfem.GridFunction.ARITHMETIC)

        # put the H1 smoothed function into a coefficient
        u_h1_coeff = mfem.GridFunctionCoefficient(u_h1)

        # create a 0-order L2 field to hold errors
        l2_0_fec = mfem.L2_FECollection(0,self.dim)
        l2_0_fes = mfem.FiniteElementSpace(self.mesh,l2_0_fec)

        # Compute elementwise "errors" between continuous and discontinuous fields
        err_gf = mfem.GridFunction(l2_0_fes);
        self.u.ComputeElementL2Errors(u_h1_coeff, err_gf);
        
        err = err_gf.GetDataArray()
        refine_list = [idx for idx, val in enumerate(err) if val > 2.e-5]
        if len(refine_list) > 0:
            self.mesh.GeneralRefinement(mfem.intArray(refine_list))
            self.fes.Update()
            self.u.Update()
            self.u.ProjectCoefficient(self.coeff)
            

    
    def render(self):
        return glvis( to_stream(self.mesh, self.u) +'keys Rcjm', 500, 500)
    
    def render_complete(self):
        return glvis( to_stream(self.mesh, self.complete) +'keys Rcjm', 500, 500)
    
    def render_orig_error(self):
        fec0 = mfem.L2_FECollection(p=0, dim=2)
        fes0 = mfem.FiniteElementSpace(self.mesh, fec0)
        el_err = mfem.GridFunction(fes0)
        self.u.ComputeElementL2Errors(self.coeff, el_err)
        return glvis( to_stream(self.mesh, el_err) +'keys Rcjm', 500, 500)
    
    def render_refined_error(self):
        
        fec0 = mfem.L2_FECollection(p=0, dim=2)
        fes0 = mfem.FiniteElementSpace(self.mesh, fec0)
        el_err1 = mfem.GridFunction(fes0)
        
        mesh0 = mfem.Mesh(self.mesh) # save coarse mesh

        refine_list = [*range(self.mesh.GetNE())]
        self.mesh.GeneralRefinement(mfem.intArray(refine_list))
        
        cft = self.mesh.GetRefinementTransforms()

        coarse_to_fine = mfem.Table()
        coarse_to_ref_type = mfem.intArray()
        ref_type_to_matrix = mfem.Table()
        ref_type_to_geom = mfem.GeometryTypeArray()

        cft.GetCoarseToFineMap(self.mesh,
                               coarse_to_fine,
                               coarse_to_ref_type,
                               ref_type_to_matrix,
                               ref_type_to_geom)
        
        self.fes.Update()
        self.u.Update()
        self.u.ProjectCoefficient(self.coeff)
        
        # compute errors on fine mesh
        fes_fine = mfem.FiniteElementSpace(self.mesh, fec0)
        el_err_fine = mfem.GridFunction(fes_fine)
        self.u.ComputeElementL2Errors(self.coeff, el_err_fine)
        
        # accumulate errors into coarse mesh
        for k in range(mesh0.GetNE()):
            els = coarse_to_fine.GetRowList(k)
            sumsq = 0.0
            for j in els:
                sumsq += el_err_fine[j]**2
            el_err1[k] = math.sqrt(sumsq)
        
        return glvis( to_stream(mesh0, el_err1) +'keys Rcjm', 500, 500)
   
    def render_error_reduction(self):
        
        fec0 = mfem.L2_FECollection(p=0, dim=2)
        fes0 = mfem.FiniteElementSpace(self.mesh, fec0)
        el_err0 = mfem.GridFunction(fes0)
        el_err1 = mfem.GridFunction(fes0)
        el_err_delta = mfem.GridFunction(fes0)
        mesh0 = mfem.Mesh(self.mesh) # save coarse mesh


        self.u.ComputeElementL2Errors(self.coeff, el_err0)
        
        nelements = self.mesh.GetNE()
        
        refine_list = [*range(self.mesh.GetNE())]
        self.mesh.GeneralRefinement(mfem.intArray(refine_list))
        
        cft = self.mesh.GetRefinementTransforms()

        coarse_to_fine = mfem.Table()
        coarse_to_ref_type = mfem.intArray()
        ref_type_to_matrix = mfem.Table()
        ref_type_to_geom = mfem.GeometryTypeArray()

        cft.GetCoarseToFineMap(self.mesh,
                               coarse_to_fine,
                               coarse_to_ref_type,
                               ref_type_to_matrix,
                               ref_type_to_geom)
        
        self.fes.Update()
        self.u.Update()
        self.u.ProjectCoefficient(self.coeff)
        
        # compute errors on fine mesh
        fes_fine = mfem.FiniteElementSpace(self.mesh, fec0)
        el_err_fine = mfem.GridFunction(fes_fine)
        self.u.ComputeElementL2Errors(self.coeff, el_err_fine)
        
        for k in range(nelements):
            els = coarse_to_fine.GetRowList(k)
            sumsq = 0.0
            for j in els:
                sumsq += el_err_fine[j]**2
            el_err1[k] = math.sqrt(sumsq)
        
        for k in range(nelements):
            el_err_delta[k] = el_err0[k] -el_err1[k]
            
        return glvis( to_stream(mesh0, el_err_delta) +'keys Rcjm', 500, 500)
        

In [98]:
env = EstimatorGameDummy(None)
g = GlobalProblem(env)

In [105]:
g.reset()
g.render()

glvis()

In [106]:
g.reinit()
g.refine_by_dg()
g.refine_by_dg()
g.render()

glvis()

In [107]:
g.reinit()
g.refine()
g.refine()
g.render()

glvis()

In [108]:
g.reinit()
g.render_error_reduction()

glvis()